In [ ]:
'''
%pip install numpy
%pip install pydicom
%pip install opencv_python
%pip install --upgrade pydicom
%pip install gdcm
%pip install pylibjpeg
%pip install matplotlib
%pip install Pillow
'''

In [ ]:
import os
import re
import numpy as np
import cv2
import pydicom
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# -------------------------------
# CONFIGURAZIONE
# -------------------------------
path = r"C:\Users\Salvatore\Desktop\Progetto"  # Modifica se serve
dicom_file = os.path.join(path, 'bmode.dcm')

operatori_disponibili = ["Mela D.", "Costarella A.", "Citriniti S."]
esami_disponibili = ["Ecocardiogramma", "Ecografia", "Ecocolordoppler", "Ecodoppler"]
reparti = ["Cardiologia", "Radiologia", "Ortopedia"]

# -------------------------------
# Controllo file DICOM
# -------------------------------
if not os.path.exists(dicom_file):
    raise FileNotFoundError(f"File DICOM non trovato: {dicom_file}")

ds = pydicom.dcmread(dicom_file)

# -------------------------------
# Inserimento informazioni paziente
# -------------------------------
ds.nome = input("Inserisci il nome del paziente: ")
ds.cognome = input("Inserisci il cognome del paziente: ")

while True:
    data_nascita = input("Inserisci la data di nascita del paziente (dd-mm-aaaa): ")
    if re.match(r"\d{2}-\d{2}-\d{4}$", data_nascita):
        ds.data_nascita = data_nascita
        break
    print("Formato data non valido.")

while True:
    cf = input("Inserisci il codice fiscale del paziente: ")
    if len(cf) == 16 and cf.isalnum():
        ds.cf = cf
        break
    print("Codice fiscale non valido.")

def scelta_menu(lista, titolo):
    print(f"\n{titolo}:")
    for i, item in enumerate(lista, 1):
        print(f"{i}. {item}")
    while True:
        scelta = input("Seleziona il numero corrispondente: ")
        if scelta.isdigit() and 1 <= int(scelta) <= len(lista):
            return lista[int(scelta)-1]
        print("Scelta non valida.")

ds.operatore = scelta_menu(operatori_disponibili, "Operatori disponibili")
ds.esame = scelta_menu(esami_disponibili, "Esami disponibili")
ds.reparto = scelta_menu(reparti, "Reparti disponibili")
ds.data = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
ds.datatext = datetime.now().strftime("%Y%m%d%H%M%S")
ds.datafolder = datetime.now().strftime("%Y%m%d")

# -------------------------------
# Prepara informazioni testo
# -------------------------------
info_text = (
    f"CF: {ds.cf}\n"
    f"Nome: {ds.nome}\n"
    f"Cognome: {ds.cognome}\n"
    f"Data di Nascita: {ds.data_nascita}\n"
    f"Reparto: {ds.reparto}\n"
    f"Operatore: {ds.operatore}\n"
    f"Esame: {ds.esame}\n"
    f"Data di Acquisizione: {ds.data}"
)

# -------------------------------
# Funzioni di utilità
# -------------------------------
def get_dynamic_font(roi_width, roi_height, max_font_size=20, min_font_size=6):
    size = int(min(max_font_size, max(min_font_size, roi_height // 20)))
    try:
        return ImageFont.truetype("arial.ttf", size)
    except:
        return ImageFont.load_default()

def draw_text_wrapped(draw, text, font, x, y, max_width, fill=255):
    lines = []
    words = text.split()
    while words:
        line = ''
        while words:
            test_line = line + (' ' if line else '') + words[0]
            bbox = draw.textbbox((0,0), test_line, font=font)
            width = bbox[2] - bbox[0]
            if width <= max_width:
                line = test_line
                words.pop(0)
            else:
                break
        lines.append(line)
    
    line_height = font.getbbox('A')[3] + 2
    for i, line in enumerate(lines):
        draw.text((x, y + i*line_height), line, font=font, fill=fill)

# -------------------------------
# Lettura dei pixel
# -------------------------------
pixel_data = ds.pixel_array
pixel_data = cv2.normalize(pixel_data, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
num_frames = pixel_data.shape[0] if len(pixel_data.shape) > 2 else 1

# -------------------------------
# Crea cartelle per output
# -------------------------------
crop_folder = os.path.join(path, 'Crop_Immagini_Esami_Dicom', ds.datafolder)
img_folder = os.path.join(path, 'Immagini_Esami_Dicom', ds.datafolder)
video_folder = os.path.join(path, 'Video_Esami_Dicom', ds.datafolder)
os.makedirs(crop_folder, exist_ok=True)
os.makedirs(img_folder, exist_ok=True)
os.makedirs(video_folder, exist_ok=True)

# -------------------------------
# Selezione ROI manuale sul primo frame
# -------------------------------
first_frame = pixel_data[0] if num_frames > 1 else pixel_data
roi = cv2.selectROI("Seleziona ROI e premi ENTER o SPAZIO", first_frame, showCrosshair=True, fromCenter=False)
cv2.destroyAllWindows()

x, y, w, h = roi
font = get_dynamic_font(w, h)

# -------------------------------
# Applica ROI a tutti i frame, salva immagini crop
# -------------------------------
file_paths = []

for i in range(num_frames):
    frame = pixel_data[i] if num_frames > 1 else pixel_data
    cropped = frame[y:y+h, x:x+w]
    img = Image.fromarray(cropped)
    draw = ImageDraw.Draw(img)
    draw_text_wrapped(draw, info_text, font, x=5, y=5, max_width=w-10, fill=255)
    img_path = os.path.join(img_folder, f"{ds.esame}_{ds.cognome}_{ds.nome}_{ds.datatext}_{i}.jpg")
    img.save(img_path)
    file_paths.append(img_path)

# -------------------------------
# Salva immagine crop principale
# -------------------------------
main_crop_path = os.path.join(crop_folder, f"{ds.esame}_{ds.cognome}_{ds.nome}_{ds.datatext}(ROI).jpg")
img_crop_main = Image.fromarray(first_frame[y:y+h, x:x+w])
draw_main = ImageDraw.Draw(img_crop_main)
draw_text_wrapped(draw_main, info_text, font, x=5, y=5, max_width=w-10, fill=255)
img_crop_main.save(main_crop_path)
print(f"Immagine crop salvata: {main_crop_path}")

# -------------------------------
# Crea video MP4 dai frame crop con testo incluso
# -------------------------------
size = (w, h)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 3
video_path = os.path.join(video_folder, f"{ds.esame}_{ds.cognome}_{ds.nome}_{ds.datatext}_text.mp4")
video_writer = cv2.VideoWriter(video_path, fourcc, fps, size)

for i in range(num_frames):
    frame = pixel_data[i] if num_frames > 1 else pixel_data
    cropped = frame[y:y+h, x:x+w]

    if len(cropped.shape) == 2:
        cropped_bgr = cv2.cvtColor(cropped, cv2.COLOR_GRAY2BGR)
    else:
        cropped_bgr = cropped.copy()

    img_pil = Image.fromarray(cropped_bgr)
    draw = ImageDraw.Draw(img_pil)
    draw_text_wrapped(draw, info_text, font, x=5, y=5, max_width=w-10, fill=(255, 255, 255))

    frame_with_text = np.array(img_pil)
    video_writer.write(frame_with_text)

video_writer.release()
print(f"Video con testo creato: {video_path}")

# -------------------------------
# Salva DICOM aggiornato
# -------------------------------
ds.save_as(os.path.join(path, f'{ds.esame}_{ds.cognome}_{ds.nome}_{ds.datatext}.dcm'))
print("DICOM aggiornato salvato correttamente!")
